A dataset with movie info is needed to create the movie_info table in the application database. I have decided to load the movies.csv file as a dataframe, clean it and save only the rows with movie IDs which are in my prepared training set. 

In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv("D:/Movie Recommendation System Project/data/raw data/movies.csv")
train_set = pd.read_csv("D:/Movie Recommendation System Project/data/data preparation/dataset frac=0.33, ratio=4/train_set.csv")

In [3]:
print(movies.info())

print("\n" + "Number of unique movie IDs in train_set: {}".format(train_set["movie_ID"].nunique()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB
None

Number of unique movie IDs in train_set: 43232


We can see above that the number of unique movie_IDs in movies is 62423, and train_set only has 43232. Rename "movieId" column in movies to "movie_ID" to create consistency between movies and train_set. Next, create a dataframe with unique movie_IDs from train_set. Then use the pandas merge() function to inner join the movies and unique train_set["movie_ID"] series based on the movie_ID value. 

In [4]:
movies.columns = ["movie_ID", "title", "genre"]

unique_movie_IDs = pd.Series(train_set["movie_ID"].unique(), name = "movie_ID").to_frame()

movies = pd.merge(left = movies, right = unique_movie_IDs, how = "inner", on = "movie_ID")

movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43232 entries, 0 to 43231
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_ID  43232 non-null  int64 
 1   title     43232 non-null  object
 2   genre     43232 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


Now we will explore the data in movies and decide how to clean it.

In [5]:
movies.sample(20)

,movie_ID,title,genre
4459,4571,Bill & Ted's Excellent Adventure (1989),Adventure|Comedy|Sci-Fi
13812,72731,"Lovely Bones, The (2009)",Crime|Drama|Fantasy|Horror|Thriller
1812,1905,Marie from the Bay of Angels (Marie Baie Des A...,Drama
27173,142961,Life Eternal (2015),Comedy|Crime|Thriller
20192,111405,Trances (1981),Documentary|Musical
24132,131164,Vietnam in HD (2011),War
32461,164395,I Am a Hero (2016),Action|Fantasy|Horror|Sci-Fi
36843,179847,The Future Perfect (2016),Comedy|Drama
12515,61638,Flu Bird Horror (2008),Horror|Thriller
41675,199706,The Photographical Congress Arrives in Lyon (1...,(no genres listed)


In [7]:
movies["genre"].value_counts()

Drama                             6275
Comedy                            3889
Documentary                       3201
(no genres listed)                2286
Comedy|Drama                      1775
                                  ... 
Comedy|Horror|Musical|Western        1
Crime|Documentary|Drama|War          1
Adventure|Comedy|Horror|Sci-Fi       1
Animation|Drama|Musical              1
Action|Children|Sci-Fi|IMAX          1
Name: genre, Length: 1532, dtype: int64

We can see that the movie titles include a year in brackets at the end. The year for each movie ID can be extracted from "title" and saved in a new column named "year". Additionally, it can be seen that there are 2286 movies with a genre value "no genres listed". These values can be renamed "Other".

In [6]:
# Create new column using the (year) at the end of each title
movies["year"] = movies["title"].astype(str).str.split().str[-1]
# Further splits to remove the brackets around the year
movies["year"] = movies["year"].str.split("(").str[-1]
movies["year"] = movies["year"].str.split(")").str[0]

# Remove the year from titles 
movies["title"] = movies["title"].astype(str).str.split().str[:-1]
movies["title"] = movies["title"].str.join(" ")

movies.sample(20)

,movie_ID,title,genre,year
36457,178299,Soldiers Pay,Documentary,2004
13078,68173,Strike (Stachka),Drama,1925
25637,136680,Battle For SkyArk,Action|Adventure|Children|Sci-Fi,2015
7833,8532,"Door in the Floor, The",Drama,2004
7016,7155,Calendar Girls,Comedy,2003
36402,177867,Borg vs McEnroe,Drama,2017
37923,184429,I Love My Wife,Comedy,1970
7655,8202,"Golden Coach, The (Le carrosse d'or)",Comedy|Drama|Romance,1953
30838,157729,Slayers Premium,Adventure|Animation|Comedy|Fantasy,2001
4660,4773,Haiku Tunnel,Comedy,2001


In [8]:
# Convert genre values to strings
movies["genre"] = movies["genre"].astype(str)

movies["genre"].loc[movies["genre"] == "(no genres listed)"] = "Other"

print(movies.loc[movies["genre"] == "(no genres listed)"].shape[0])
print(movies.loc[movies["genre"] == "Other"].shape[0])

0
2286


C:\Users\James\anaconda3\envs\tensor\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
movies.to_csv("D:/Movie Recommendation System Project/data/data preparation/dataset frac=0.33, ratio=4/movie_info.csv")